In [1]:
import pandas as pd
import re
from collections import Counter
import spacy
import wikipediaapi
wiki = wikipediaapi.Wikipedia('student_homework')

In [2]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']
win_data = pd.read_csv('wins.csv')['text']
spacy_data = pd.read_csv('spacy_info.csv')

In [4]:
nlp = spacy.load('en_core_web_sm')

def extract_person_names(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == 'PERSON']

In [14]:
def extract_and(text):
    match = re.match(r"\b([A-Z][a-z]+(?: [A-Z][a-z]+)?) and ([A-Z][a-z]+(?: [A-Z][a-z]+)?)\b", text)
    return match.group(0) if match else None

In [16]:
all_pairs = cleaned_data.apply(extract_and).dropna()

In [17]:
name_counts = Counter(all_pairs)

In [32]:
name_counts.most_common(1)[0][0].split(" and ")

['Tina Fey', 'Amy Poehler']

In [80]:
host_keywords = r'\b(host|hosts|hosting)\b'

host_data = cleaned_data[
    cleaned_data.str.contains(host_keywords, case=False, na=False)
]

def extract_person_names(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == 'PERSON']

C:\Users\rockm\AppData\Local\Temp\ipykernel_12312\4051002869.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cleaned_data.str.contains(host_keywords, case=False, na=False)


In [81]:
host_data = host_data[:int(0.05*len(host_data))].apply(extract_person_names)

all_names = [name for names_list in host_data for name in names_list]

name_counts = Counter(all_names)

In [85]:
def lookup_wikipedia(name):
   page = wiki.page(name)

   if page.exists():
      return re.search("born", page.summary, re.IGNORECASE) != None 
   else:
      return False

In [115]:
hosts = []

i = 1
potential = name_counts.most_common()
while len(hosts) < 2:
    curr = potential[i][0]
    if lookup_wikipedia(curr):
        hosts.append(curr)
    i += 1

In [116]:
hosts

['Amy Poehler', 'Tina Fey']